In [2]:
import requests
import re
from bs4 import BeautifulSoup
import wikipedia
import wikipediaapi



In [25]:
BASE_URL = "https://en.wikipedia.org/wiki/List_of_sculptors"

def get_sculptors(url):
    response = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    sculptors = []
    list_elements = soup.find('div', class_='div-col')
    if list_elements:
        list_sculptors = list_elements.find_all('ul')
        for sculptor in list_sculptors:
            sculptor = sculptor.text.strip().split('\n')
            
            sculptors.extend(sculptor)
    return sculptors


sculptors = get_sculptors(BASE_URL)
print(sculptors)




['Wäinö Aaltonen (1894–1966), Finland', 'Johannes Josephus Aarts (1871–1934), Netherlands', 'Magdalena Abakanowicz (1930–2017), Poland', 'Elfriede Abbe (1919–2012), US', 'Louise Abbéma (1853–1927), France', 'Abed Abdi (عبد عابدي, born 1942), Palestine/Israel', 'Pablita Abeyta (1953–2017), US', 'Antonio Abondio (1538–1591), Italy', 'Per Abramsen (1941–2018), Netherlands', 'Julio Abril (1911–1979), Colombia', 'Jane Ackroyd (born 1957), England', 'József Ács (1931–2023), Hungary', 'Lambert-Sigisbert Adam (1700–1759), France', 'Nicolas-Sébastien Adam (1705–1778), France', 'Alice Adams (born 1930), US', 'George Gammon Adams (1821–1898), England', 'Amandus Adamson (1855–1929), Estonia', 'Yaacov Agam (יעקב אגם, born 1928), Palestine/Israel', 'Agasias, son of Dositheus (4th century BC), Ephesus', 'Peter Agostini (1913–1993), US', 'Agostino di Duccio (1418 – c. 1481), Italy', 'Agostino da Siena (c.\u20091285 – c.\u20091347), Italy', "Gian Francesco d'Agrate (1489 – post-1563), Italy", 'Armen Ag

In [21]:
def get_biographies(sculptors):
    biographies = []
    for sculptor in sculptors:
        # if sculptor.ns == wikipediaapi.Namespace.MAIN:
            try:
                page = wikipedia.page(wikipedia.search(sculptor)[0])
                biography = page.summary
                biographies.append((sculptor, biography))
            except wikipedia.exceptions.DisambiguationError as e:
            # Skip the problematic page and continue to the next sculptor (in case of disambiguation error)
                print(f"DisambiguationError for {sculptor}: {e}")
                continue
            except wikipedia.exceptions.PageError as e:
                print(e)
    return biographies

biographies = get_biographies(sculptors)
print(biographies)

In [28]:
import pandas as pd
biography = pd.DataFrame(biographies, columns=['Sculptor', 'Biography'])
biography.to_csv('biography.csv', index=False)

In [29]:
df = pd.read_csv('biography.csv')
df.head()

,Sculptor,Biography
0,"Wäinö Aaltonen (1894–1966), Finland",Wäinö Valdemar Aaltonen (8 March 1894 – 30 May...
1,"Johannes Josephus Aarts (1871–1934), Netherlands","Johannes Josephus Aarts (18 August 1871, in Th..."
2,"Magdalena Abakanowicz (1930–2017), Poland",Magdalena Abakanowicz (Polish pronunciation: [...
3,"Elfriede Abbe (1919–2012), US",This is a list of sculptors – notable people k...
4,"Louise Abbéma (1853–1927), France",Louise Abbéma (30 October 1853 – 29 July 1927)...


In [6]:
url2 = "https://en.wikipedia.org/wiki/List_of_computer_scientists"
# cs = get_sculptors(url2)

In [31]:
print(get_biographies(cs))

[('List of computing people', 'This is a list of people who are important or notable in the field of computing, but who are not primarily computer scientists or programmers.'), ('List of important publications in computer science', 'This is a list of important publications in computer science, organized by field. Some reasons why a particular publication might be regarded as important:\n\nTopic creator – A publication that created a new topic\nBreakthrough – A publication that changed scientific knowledge significantly\nInfluence – A publication which has significantly influenced the world or has had a massive impact on the teaching of computer science.'), ('List of Jewish American computer scientists', 'This is a list of notable Jewish American computer scientists.  For other Jewish Americans, see Lists of Jewish Americans.\n\nScott Aaronson, quantum computing\nHal Abelson, artificial intelligence\nLeonard Adleman, RSA cryptography, DNA computing, Turing Award (2002)\nAdi Shamir, RSA 

In [4]:
print(wikipedia.search('Sculptor'))

['Sculpture', 'The Sculptor', 'Sculptor (disambiguation)', 'Sculptor Group', 'Sculptor Wall', 'Sculptor (constellation)', 'Jakanachari', 'Sculptor Capital Management', 'John Bunting (sculptor)', 'Sculptor Galaxy']


In [18]:
def get_sculptors(url):
    response = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')
    sculptors = []
    list_elements = soup.find('div', class_='mw-body-content')
    if list_elements:
        list_sculptors = list_elements.find_all('ul')
        for sculptor in list_sculptors:
            sculptor = sculptor.text.strip().split('\n')
            
            sculptors.extend(sculptor)
    return sculptors


sculptors = get_sculptors(url2)
cleaned_sculptors = []
for item in sculptors:
     pattern = r'^.*?(?=\s*–)'
     match = re.match(pattern, item)
     if match:
         cleaned_sculptor = match.group().strip()
         cleaned_sculptors.append(cleaned_sculptor)
print(cleaned_sculptors)

['Atta ur Rehman Khan', 'Wil van der Aalst', 'Scott Aaronson', 'Rediet Abebe', 'Hal Abelson', 'Serge Abiteboul', 'Samson Abramsky', 'Leonard Adleman', 'Manindra Agrawal', 'Luis von Ahn', 'Alfred Aho', 'Frances E. Allen', 'Gene Amdahl', 'David P. Anderson', 'Lisa Anthony', 'Andrew Appel', 'Cecilia R. Aragon', 'Bruce Arden', 'Angie Jones', 'Sanjeev Arora', 'Winifred "Tim" Alice Asprey', 'John Vincent Atanasoff', 'Shakuntala Atre', 'Lennart Augustsson', 'Charles Babbage (1791', 'Charles Bachman', 'Roland Carl Backhouse', 'John Backus', 'David F. Bacon', 'Anthony James Barr', 'Jean Bartik (1924', 'Friedrich L. Bauer', 'Rudolf Bayer', 'Gordon Bell (born 1934)', 'Steven M. Bellovin', 'Cecilia Berdichevsky (1925', 'Tim Berners-Lee', 'Daniel J. Bernstein', 'Jeff Bezos', 'Dines Bjørner', 'Gerrit Blaauw', 'Dorothy Blum', 'Lenore Blum', 'Manuel Blum', 'Barry Boehm', 'Corrado Böhm', 'Jeff Bonwick', 'Grady Booch', 'George Boole', 'Andrew Booth', 'Kathleen Booth', 'Anita Borg (1949', 'Bert Bos', 'Mi

In [22]:
print(get_biographies(cleaned_sculptors))

Page id "scott aaron son" does not match any pages. Try another id!
Page id "luis von ah" does not match any pages. Try another id!
Page id "john bakus" does not match any pages. Try another id!
Page id "jean bart" does not match any pages. Try another id!
Page id "tim benners lee" does not match any pages. Try another id!
Page id "jeff be os" does not match any pages. Try another id!
Page id "manuel bloom" does not match any pages. Try another id!
Page id "george poole" does not match any pages. Try another id!


/home/oumar/Projects/UE-803/venv/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/oumar/Projects/UE-803/venv/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError for Andrew Booth: "Andrew Booth" may refer to: 
Andy Booth
Andy Booth (racing driver)
Andy Booth (rugby union)
Andrew Donald Booth
Andrew Booth (soccer)
Andrew Booth Jr.
The Cassandra Complex
character in the television show Monarch of the Glen
Page id "sergey bin" does not match any pages. Try another id!
DisambiguationError for David J. Brown: "David J. Brown" may refer to: 
David J. Brown (cricketer)
David W. J. Brown
David J. Brown, Sr.
David J. Brown (computer scientist)
David Jay Brown
David Brown (disambiguation)
Page id "fred books" does not match any pages. Try another id!
DisambiguationError for Michael Butler: "Michael Butler" may refer to: 
Michael Butler (computer scientist)
Michael Butler (diplomat)
Michael Butler (musician)
Michael Butler (politician)
Michael Butler (producer)
Michael Butler (soccer)
Michael E. Butler
Michael Butler (Missouri politician)
Mick Butler (Dublin hurler)
Mick Butler (footballer)
Mick Butler (Wexford hurler)
Mike Butler (Ame